In [ ]:
import numpy as np
import math
import copy


# 関数

def calculate_exp_sd(RS, s, d, parameter_p, cost_sr, cost_rs, value, expect_minimum_sd_before):
    Exp_expect_cost_sd = {}
    for rs in RS:
        Exp_expect_cost_sd[rs] = math.exp( -parameter_p * ( cost_sr.get((s,rs[0])) + cost_rs.get(rs) - value.get(rs) + expect_minimum_sd_before.get((rs[0],d)) ) )
        
    return Exp_expect_cost_sd


def calculate_exp_od(RS, o, d, parameter_p, cost_or, cost_rs, value, expect_minimum_sd):
    Exp_expect_cost_od = {}
    for rs in RS:
        Exp_expect_cost_od[rs] = math.exp( -parameter_p * ( cost_or.get((o,rs[0])) + cost_rs.get(rs) - value.get(rs) + expect_minimum_sd.get((rs[1],d)) ) )
        
    return Exp_expect_cost_sd

    
def delta(r_s, rs):
    if rs == r_s:
        return 1
    else:
        return 0


def calculate_sum_probability_sd(P_probability_sd, s, d, rs, RS, partial_differentiation_before):
    sum_probability_sd = 0
    for r_s in RS:
        sum_probability_sd += P_probability_sd.get((s,d,r_s)) * ( delta(r_s, rs) - partial_differentiation_before.get((r_s[1],d)) )   # from def delta
        
    return sum_probability_sd


def calculate_sum_probability_od(P_probability_od, o, d, rs, RS, partial_differentiation_sd):
    sum_probability_od = 0
    for r_s in RS:
        sum_probability_od += P_probability_od.get(o,d,r_s)) * ( -delta(r_s, rs) + partial_differentiation_sd.get((r_s[1],d)) )
        
    return sum_probability_od


def calculate_sum_expect_cost_od_driver(O, D, num_driver_od, expect_minimum_od):
    sum_expect_cost_od_driver = {}
    for o in O:
        for d in D:
            sum_expect_cost_od_driver[(o,d)] = num_driver_od.get((o,d)) * expect_minimum_od.get((o,d))
            
    return sum_expect_cost_od_driver


def calculate_sum_expect_cost_rs_shipper(R, S, num_shipper_rs, fixed_expected_minimum_cost_rs):
    sum_expect_cost_rs_shipper = {}
    for r in R:
        for s in S:
            sum_expect_cost_rs_shipper[(r,s)] = num_shipper_rs.get((r,s)) * fixed_expected_minimum_cost_rs.get((r,s))
            
    return sum_expect_cost_rs_shipper


def calculate_partial_differentiation(O, D, rs, num_driver_od, partial_differentiation_od):
    partial_differentiation = {}
    for o in O:
        for d in D:
            partial_differentiation[(o,d,rs)] = num_driver_od.get((o,d)) * partial_differentiation_od.get((o,d,rs))

    sum_partial_differentiation = sum( partial_differentiation )
    partial_differentiation.clear()
    
    return sum_partial_differentiation
                                          



# サブアルゴリズム本体
def MCA(value, value_0, run_limit, cost_or, cost_rs, cost_sd, cost_rs, 
        parameter_p, parameter_c, 
        O, R, S, D, RS, 
        num_shipper_rs, fixed_cost_do_shipper_rs, fixed_cost_dont_shipper_rs):

    '''
    value : 実行価格 {('r','s'):value}
    value_0 : 非委託価格　{('r','s'):value}
    run_limit : 実行回数の上限 [1]
    
    cost_or : or間のコスト {(o,r):cost}
    cost_rs : rs間のコスト {(r,s):cost}
    cost_sd : sd間のコスト {(s,d):cost}
    cost_sr : sr間のコスト {(s,r):cost}

    parameter_p : 配送者のロジットパラメータ [1]
    parameter_c : 荷主のロジットパラメーター [1]

    O : 配送者の起点集合 {'o'}
    R : タスクの起点集合 {'r'}
    S : タスクの終点集合 {'s'}
    D : 配送者の終点集合 {'d'}
    RS : タスクのODペア集合 {('r','s')}

    num_driver_od : od配送者数 {('o','d'):num}
    num_shipper_rs : rs荷主数 {('r','s'):num}
    fixed_cost_do_shipper_rs : rs荷主の委託確定費用 {('r','s'):cost_do}
    fixed_cost_dont_shipper_rs : rs荷主の非委託確定費用 {('r','s'):cost_dont}
    '''

    # 初期値設定
    expect_minimum_sd_before = copy.deepcopy(cost_sd)   # expect_minimum_sd_bofore : 一つ前での期待最小費用 
    partial_differentiation_before = {(s, d): 0 for s in S for d in D}   # partial_differentiation_before : 一つ前での期待最小費用の偏微分

    
    # 辞書生成 (s*dループ用)
    Z_expect_cost_sd = {}   # Z_sd^(i)
    P_probability_sd = {}   # P_sd^(i)
    partial_differentiation_sd = {}   # partial(mu_sd^(i))
    expect_minimum_sd = {}   # mu_sd^(i)

    fixed_expected_minimum_cost_rs = {}   # V_rs(v_rs)
    num_do_shipper_rs = {}   # z_rs^1(v_rs)
    
    # 

    # アルゴリズム
    for i in range(1,run_limit-1):
        
        if i > 1:   # i > 1の時一つ前のループでの辞書更新
            # mu_sd^(i)の辞書更新
            expect_minimum_sd_before.clear()
            expect_minimum_sd_before = copy.deepcopy(expect_minimum_sd)   

            # partial(mu_sd^(i))の辞書更新
            partial_differentiation_before.clear()
            partial_differentiation_before = copy.deepcopy(partial_differentiation)

            # その他の辞書更新
            Z_expect_cost_sd.clear()   # Z_sd^(i)
            P_probability_sd.clear()   # P_sd^(i)
        

        # s*dループ
        for s in S:
            for d in D:
                # あらかじめ計算
                exp_expect_cost_sd = calculate_exp_sd(RS, s, d, parameter_p, cost_sr, cost_rs, value, expect_minimum_sd_before)   # from def calculate_exp_sd 

                # 計算
                Z_expect_cost_sd[(s,d)] = math.exp( -parameter_p * cost_sd.get((s,d)) ) + sum( exp_expect_cost_sd.values() )   # Compute Z_sd^(i)
                
                for rs in RS:
                    P_probability_sd[(s,d,rs)] = exp_expect_cost_sd.get(rs) / Z_expect_cost_sd.get((s,d))   # Compute P_sd^(i) 
                    
                for rs in RS:
                    partial_differentiation_sd[(s,d,rs)] = - calculate_sum_probability_sd(P_probality_sd, s, d, rs, RS, partial_differentiation_before)   # Compute partial(mu_sd^(i))  from def calculate_sum_probability_sd
                    
                expect_minimum_sd[(s,d)] = -1 / parameter_p * math.log( math.exp( -parameter_p * cost.get((s,d)) ) + sum( exp_expect_cost_sd.values() ) )   # Compute mu_sd^(i)


        
    
    # 荷主の期待最小費用と委託数の計算
    for rs in RS:
        fixed_expected_minimum_cost_rs[rs] = -1 / parameter_c * math.log( math.exp( -parameter_c * ( fixed_cost_dont_shipper_rs.get(rs) + value_0.get(rs) )) + math.exp ( -parameter_c * ( fixed_cost_do_shipper_rs.get(rs) + value.get(rs) ) ) )   # Compute V_rs(v_rs)
        
        num_do_shipper_rs[rs] = num_shipper_rs.get(rs) * math.exp( -parameter_c * ( fixed_cost_do_shipper_rs[rs] + value[rs])) / ( math.exp( -parameter_c * ( fixed_cost_do_shipper_rs[rs] + value[rs])) + math.exp( -parameter_c *  fixed_cost_dont_shipper_rs[rs]))   # Compute z_rs^1(v_rs)


    # 辞書生成 (o*dループ用)
    Z_expect_cost_od = {}
    P_probability_od = {}
    partial_differentiation_od = {}
    expect_minimum_od = {}
    
    # o*dループ
    for o in O:
        for d in D:
            # あらかじめ計算
            exp_expect_cost_od = calculate_exp_od(RS, o, d, parameter_p, cost_or, cost_rs, value, expect_minimum_sd)   # from calculate_exp_od

            # 計算
            Z_expect_cost_od = math.exp( -parameter_p * cost_od.get((o,d)) ) + sum( exp_expect_cost_od.values() )   # Compute Z_od^(n)

            for rs in RS:
                P_probability_od[(o,d,rs)] = exp_expect_cost_od.get(rs) / Z_expect_cost_od.get((o,d))   # Compute P_od^(n)

            for rs in RS:
                partial_differentiation_od[(o,d,rs)] = - calculate_sum_probability_od(P_probability_od, o, d, rs, RS, partial_differentiation_sd)   # Compute partial(mu_od^(n)) from def calculate_sum_probability_od

            expect_minimum_od[(o,d)] = -1 / parameter_p * math.log( math.exp( -parameter_p * cost_od.get((o,d)) ) + sum( exp_expect_cost_od.values() ) )   # Compute mu_od^(n)


    # MC & gradientMC 導出


    # MC
    # 双対問題の各項sumの中身計算
    sum_expect_cost_od_driver = calculate_sum_expect_cost_od_driver(O, D, num_driver_od, expect_minimum_od)   # Compute bar(y)_od * mu_od^(n) from def calculate_sum_expect_cost_od_driver
    sum_expect_cost_rs_shipper = calculate_sum_expect_cost_rs_shipper(R, S, num_shipper_rs, fixed_expected_minimum_cost_rs) # Compute bar(z)_rs * V_rs(v_rs) from def calculate_sum_expect_cost_rs_shipper

    # solve MC
    dual_obj = sum( sum_expect_cost_od_driver.values() ) + sum( sum_expect_cost_rs_shipper.values() )   # Compute MC from def calculate_sum_expect_cost_od_driver & def calculate_sum_expect_cost_rs_shipper

    
    # gradientMC
    # 辞書生成
    dual_grad = {}

    # solve gardientMC
    for rs in RS:
        dual_grad[rs] = calculate_partial_differentiation(O, D, rs, num_driver_od, partial_differentiation_od) + num_do_shipper_rs.get(rs)

    
    return dual_obj, dual_grad

